## Game of Nim
In this Python Notebook we will explore different ways of representing the *Game of Nim** in Python with hopes of extrapolating an optimal strategy to playing the game.

In [757]:
# import required libraries
import random as r
from collections import Counter
import numpy as np
import copy

## Random Two Player Game 

In [758]:
# Setup the amount in each pile
piles = [10,14,18]

In [759]:
# random approach

while True:

  selectable1 = False
  selectable2 = False


  # Player 1's Turn

  # Check to see if there is one remaing pile at the end of Player 1's Turn
  if piles.count(0) == 2:
    player1 = piles.index(max(piles)) + 1
    amount1 = max(piles)

  else:
    while selectable1 == False:
      player1 = r.randint(1,3)

      if piles[player1-1] != 0:
        selectable1 = True
    amount1 = r.randint(1,piles[player1 - 1])

  piles[player1 - 1] = piles[player1 - 1] - amount1
  
  # Display Player 1's actions and resulting game state
  print("Player 1: takes " + str(amount1) + " from pile " + str(player1))
  print("The current state is: " + str(piles))
  
  # Check if Player 1 wins by end of their turn
  if sum(piles) == 0:
    print("Player 1 Wins!")
    break

  # Player 2's Turn

  # Check to see if there is one remaing pile at the beginning of Player 2's turn
  if piles.count(0) == 2:
    player2 = piles.index(max(piles)) + 1
    amount2 = max(piles)
  else:
    while selectable2 == False:
      player2 = r.randint(1,3)

      if piles[player2-1] != 0:
        selectable2 = True
    amount2 = r.randint(1,piles[player2 - 1])
  piles[player2 - 1] = piles[player2 - 1] - amount2

  # Display Player 2's actions and resulting game state
  print("Player 2: takes " + str(amount2) + " from pile " + str(player2))
  print("The current state is: " + str(piles))

  # Check if Player 2 wins by end of their turn
  if sum(piles) == 0:
    print("Player 2 Wins!")
    break

Player 1: takes 5 from pile 2
The current state is: [10, 9, 18]
Player 2: takes 9 from pile 3
The current state is: [10, 9, 9]
Player 1: takes 6 from pile 1
The current state is: [4, 9, 9]
Player 2: takes 9 from pile 3
The current state is: [4, 9, 0]
Player 1: takes 2 from pile 2
The current state is: [4, 7, 0]
Player 2: takes 4 from pile 1
The current state is: [0, 7, 0]
Player 1: takes 7 from pile 2
The current state is: [0, 0, 0]
Player 1 Wins!


## AI Approach
Here we will use a *Minimax* algorithm to evaluate all the possible board states from a given present board state, then the AI will chose the next board state with the best return.

In [760]:
# enumerate over all possible actions at current board state
def actions(s):
    return [(i,j) for j in range(len(s)) if s[j] != 0 for i in range(1,s[j]+1)]

In [761]:
# apply chosen action on current board state
def result(s,a):

    # get action pile and amount
    pile = a[1]
    amount = a[0]

    # apply action and return resulting board state
    s_copy = s.copy()
    s_copy[pile] = s_copy[pile] - (amount)
    
    return s_copy
    

In [762]:
# check if there is a winner (is checked at end of turn)
def checkWinner(s,turn):
    # check if game is ongoing
    if sum(s) != 0:
        return None

    # assume AI goes first

    # case that human wins
    elif turn % 2 == 0:
        return -1
    
    # case that AI wins
    else:
        return 1


In [763]:
# implement Minimax algorithm
def minimax(s, turn, isMaxamizing):

    s_copy = s.copy()
    moves = actions(s_copy)

    if turn != 1:
        prevTurn = turn - 1
    
    else:
        prevTurn = 1
    # base case
    if checkWinner(s, prevTurn) != None:
        return checkWinner(s,prevTurn)
    
    # recursive case

    # maximizing case
    elif(isMaxamizing):
        score = -np.inf
        for move in moves:
            resulting = result(s_copy, move)
            score = max(score, minimax(resulting, turn+1,False))
            
        return score
    
    # minimizing case
    else:
        score = np.inf
        for move in moves:
            resulting = result(s_copy,move)
            score = min(score, minimax(resulting, turn+1, True))

        return score


In [764]:
# play the game of nim
s = [5,5,5]
turn = 1
print(s)
while checkWinner(s, turn) == None:

    # AI goes first
    moves = actions(s)
    score = 1 # TODO: Figure out why score must be 1 and fails otherwise?
    bestMove = None
    resulting = 0
    for move in moves:
        test = result(s, move)
        #print(test)
        resulting = minimax(test,turn, True)
        #print(resulting)
        if resulting < score: # TODO: why is the minimax algo off by one player turn?
            bestMove = move

    print("AI takes " + str(bestMove[0]) + " from " + str(bestMove[1] + 1))
    s = result(s, bestMove)
    print(s)

    if checkWinner(s, turn) == 1:
        print("AI wins!")
        break
    
    turn += 1
    # Person goes second
    pile = int(input("Please enter the pile: "))
    amount = int(input("Please enter the amount: "))

    s = result(s,(amount,pile-1))

    print(s)
    if checkWinner(s, turn) == -1:
        print("You win!")
        break

    turn += 1

[5, 5, 5]
AI takes 5 from 3
[5, 5, 0]
[2, 5, 0]
AI takes 3 from 2
[2, 2, 0]
[1, 2, 0]
AI takes 1 from 2
[1, 1, 0]
[0, 1, 0]
AI takes 1 from 2
[0, 0, 0]
AI wins!
